In [85]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import requests
import csv
import datetime as dt  
%matplotlib inline
import seaborn as sns
%pylab inline
pd.options.display.mpl_style = 'default'
import matplotlib
matplotlib.style.use('ggplot')
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns',200)

Populating the interactive namespace from numpy and matplotlib


In [86]:
# Read files
loanData1 = pd.read_csv('/Users/chrispiccin/metis_class_project/data/Lending_Club/LoanStats3a.csv')
loanData2 = pd.read_csv('/Users/chrispiccin/metis_class_project/data/Lending_Club/LoanStats3b.csv')
loanData3 = pd.read_csv('/Users/chrispiccin/metis_class_project/data/Lending_Club/LoanStats3c.csv')
loanData4 = pd.read_csv('/Users/chrispiccin/metis_class_project/data/Lending_Club/LoanStats3d.csv')

In [87]:
# Combine files into a singel DataFrame
LoanData = pd.concat([loanData1, loanData2, loanData3, loanData4], ignore_index=True)

In [88]:
#Starting Shape of Dataframe
LoanData.shape

(887383, 111)

In [89]:
# Only some data is avail prior to loan selection.  Drop unused columns
LoanData.drop(LoanData[['funded_amnt','funded_amnt_inv','installment','emp_title','pymnt_plan','desc','title','initial_list_status','out_prncp','out_prncp_inv','total_pymnt','total_pymnt_inv','total_rec_prncp','total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee','last_pymnt_d','last_pymnt_amnt','next_pymnt_d','last_credit_pull_d','collections_12_mths_ex_med','policy_code','annual_inc_joint','dti_joint','verification_status_joint','open_acc_6m','open_il_6m','open_il_12m','open_il_24m','mths_since_rcnt_il','total_bal_il','il_util','open_rv_12m','open_rv_24m','max_bal_bc','all_util','total_rev_hi_lim','inq_fi','total_cu_tl','inq_last_12m','acc_open_past_24mths','avg_cur_bal','bc_open_to_buy','bc_util','chargeoff_within_12_mths','delinq_amnt','mo_sin_old_il_acct','mo_sin_old_rev_tl_op','mo_sin_rcnt_rev_tl_op','mo_sin_rcnt_tl','mort_acc','mths_since_recent_bc','mths_since_recent_bc_dlq','mths_since_recent_inq','mths_since_recent_revol_delinq','num_accts_ever_120_pd','num_actv_bc_tl','num_actv_rev_tl','num_bc_sats','num_bc_tl','num_il_tl','num_op_rev_tl','num_rev_accts','num_rev_tl_bal_gt_0','num_sats','num_tl_120dpd_2m','num_tl_30dpd','num_tl_90g_dpd_24m','num_tl_op_past_12m','pct_tl_nvr_dlq','percent_bc_gt_75','pub_rec_bankruptcies','tax_liens','tot_hi_cred_lim','total_bal_ex_mort','total_bc_limit','total_il_high_credit_limit']], axis=1, inplace=True)

In [90]:
LoanData.shape

(887383, 33)

In [91]:
# Count of Loans by status
LoanData.loan_status.value_counts()

Current                                                555383
Fully Paid                                             251122
Charged Off                                             56773
Late (31-120 days)                                      11763
In Grace Period                                          7146
Late (16-30 days)                                        2165
Does not meet the credit policy. Status:Fully Paid       1988
Does not meet the credit policy. Status:Charged Off       761
Default                                                   281
Name: loan_status, dtype: int64

In [92]:
# Remove all loans that are still in progress
LoanData = LoanData[[x in ['Charged Off','Default','Fully Paid','Does not meet the credit policy.  Status:Charged Off','Does not meet the credit policy.  Status:Default','Does not meet the credit policy.  Status:Fully Paid'] for x in LoanData['loan_status']]]
LoanData.index = range(len(LoanData))

In [93]:
# Count of closed loans by status
LoanData.loan_status.value_counts()

Fully Paid     251122
Charged Off     56773
Default           281
Name: loan_status, dtype: int64

In [94]:
# Group defaulted and charged off loans, 1 == good loans, 0 == bad loans
LoanData['loan_status'] = [1 if x == "Fully Paid" else 0 for x in LoanData['loan_status']]

In [95]:
# More than 22% of closed loans are bad loans
LoanData.loan_status.value_counts()

1    251122
0     57054
Name: loan_status, dtype: int64

In [96]:
LoanData.head(10)

,id,member_id,loan_amnt,term,int_rate,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,url,purpose,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,mths_since_last_major_derog,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal
0,1077501,1296599.0,5000.0,36 months,10.65%,B,B2,10+ years,RENT,24000.0,Verified,Dec-2011,1,https://lendingclub.com/browse/loanDetail.acti...,credit_card,860xx,AZ,27.65,0.0,Jan-1985,1.0,NaN,NaN,3.0,0.0,13648.0,83.7%,9.0,NaN,INDIVIDUAL,0.0,NaN,NaN
1,1077430,1314167.0,2500.0,60 months,15.27%,C,C4,< 1 year,RENT,30000.0,Source Verified,Dec-2011,0,https://lendingclub.com/browse/loanDetail.acti...,car,309xx,GA,1.00,0.0,Apr-1999,5.0,NaN,NaN,3.0,0.0,1687.0,9.4%,4.0,NaN,INDIVIDUAL,0.0,NaN,NaN
2,1077175,1313524.0,2400.0,36 months,15.96%,C,C5,10+ years,RENT,12252.0,Not Verified,Dec-2011,1,https://lendingclub.com/browse/loanDetail.acti...,small_business,606xx,IL,8.72,0.0,Nov-2001,2.0,NaN,NaN,2.0,0.0,2956.0,98.5%,10.0,NaN,INDIVIDUAL,0.0,NaN,NaN
3,1076863,1277178.0,10000.0,36 months,13.49%,C,C1,10+ years,RENT,49200.0,Source Verified,Dec-2011,1,https://lendingclub.com/browse/loanDetail.acti...,other,917xx,CA,20.00,0.0,Feb-1996,1.0,35.0,NaN,10.0,0.0,5598.0,21%,37.0,NaN,INDIVIDUAL,0.0,NaN,NaN
4,1075269,1311441.0,5000.0,36 months,7.90%,A,A4,3 years,RENT,36000.0,Source Verified,Dec-2011,1,https://lendingclub.com/browse/loanDetail.acti...,wedding,852xx,AZ,11.20,0.0,Nov-2004,3.0,NaN,NaN,9.0,0.0,7963.0,28.3%,12.0,NaN,INDIVIDUAL,0.0,NaN,NaN
5,1072053,1288686.0,3000.0,36 months,18.64%,E,E1,9 years,RENT,48000.0,Source Verified,Dec-2011,1,https://lendingclub.com/browse/loanDetail.acti...,car,900xx,CA,5.35,0.0,Jan-2007,2.0,NaN,NaN,4.0,0.0,8221.0,87.5%,4.0,NaN,INDIVIDUAL,0.0,NaN,NaN
6,1071795,1306957.0,5600.0,60 months,21.28%,F,F2,4 years,OWN,40000.0,Source Verified,Dec-2011,0,https://lendingclub.com/browse/loanDetail.acti...,small_business,958xx,CA,5.55,0.0,Apr-2004,2.0,NaN,NaN,11.0,0.0,5210.0,32.6%,13.0,NaN,INDIVIDUAL,0.0,NaN,NaN
7,1071570,1306721.0,5375.0,60 months,12.69%,B,B5,< 1 year,RENT,15000.0,Verified,Dec-2011,0,https://lendingclub.com/browse/loanDetail.acti...,other,774xx,TX,18.08,0.0,Sep-2004,0.0,NaN,NaN,2.0,0.0,9279.0,36.5%,3.0,NaN,INDIVIDUAL,0.0,NaN,NaN
8,1070078,1305201.0,6500.0,60 months,14.65%,C,C3,5 years,OWN,72000.0,Not Verified,Dec-2011,1,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,853xx,AZ,16.12,0.0,Jan-1998,2.0,NaN,NaN,14.0,0.0,4032.0,20.6%,23.0,NaN,INDIVIDUAL,0.0,NaN,NaN
9,1069908,1305008.0,12000.0,36 months,12.69%,B,B5,10+ years,OWN,75000.0,Source Verified,Dec-2011,1,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,913xx,CA,10.78,0.0,Oct-1989,0.0,NaN,NaN,12.0,0.0,23336.0,67.1%,34.0,NaN,INDIVIDUAL,0.0,NaN,NaN


In [97]:
# Clean up 'member_id'
LoanData['member_id'] = LoanData['member_id'].astype(str)
LoanData['member_id'] = [x[:7] for x in LoanData['member_id']]

In [98]:
# Clean up interest rate
LoanData['int_rate'] = LoanData['int_rate'].str.replace('%', '')
LoanData['int_rate'] = LoanData['int_rate'].astype(float)

In [99]:
# Clean up 'revol_util'
LoanData['revol_util'] = LoanData['revol_util'].str.replace('%', '')
LoanData['revol_util'] = LoanData['revol_util'].astype(float)
LoanData['revol_util'] = [-1 if pd.isnull(x) else x for x in LoanData['revol_util']]

In [100]:
# clean up 'issue_d'
LoanData['issue_month'] = 0
LoanData['issue_year'] = 0
LoanData['issue_month'] = [pd.to_datetime(x).month for x in LoanData['issue_d']]
LoanData['issue_year'] = [pd.to_datetime(x).year for x in LoanData['issue_d']]

In [101]:
# Clean up zip_code
LoanData['zip_code'] = [x[:3] for x in LoanData['zip_code']]
LoanData['zip_code'] = LoanData['zip_code'].astype(float)
LoanData['zip_code'] = LoanData['zip_code'].astype(int)

In [102]:
# Clean up 'yrs_since_first_cr_line'
LoanData['first_cr_line'] = 0
LoanData['first_cr_line'] = [date.today().year-pd.to_datetime(x).year for x in LoanData['earliest_cr_line']]

NameError: name 'date' is not defined

In [103]:
LoanData.emp_length.value_counts().sort_index()

1 year       20471
10+ years    94820
2 years      28556
3 years      24904
4 years      19310
5 years      21539
6 years      17625
7 years      16938
8 years      14517
9 years      11715
< 1 year     25294
n/a          12487
Name: emp_length, dtype: int64

In [104]:
# Convert 'emp_length' to categorical
emp_years = dict(zip(LoanData['emp_length'].unique(), np.arange(LoanData['emp_length'].nunique())))
LoanData['emp_length'] = LoanData['emp_length'].map(lambda x: emp_years[x])

In [105]:
LoanData.emp_length.value_counts().sort_index()

0     94820
1     25294
2     24904
3     11715
4     19310
5     21539
6     20471
7     17625
8     28556
9     16938
10    14517
11    12487
Name: emp_length, dtype: int64

In [106]:
# 3 bits for binary conversion
LoanData.grade.value_counts()

B    92023
C    80327
A    50635
D    49655
E    23828
F     9340
G     2368
Name: grade, dtype: int64

In [52]:
# Convert 'grade' to categorical
LoanData = LoanData[pd.isnull(LoanData['grade']) == 0]
LoanData.index = range(len(LoanData))
grades_cat = pd.get_dummies(LoanData['grade'])
LoanData = pd.concat([LoanData, grades_cat], axis=1)

In [53]:
# 6 bits for binary conversion
LoanData.sub_grade.value_counts().sort_index()

A1     7141
A2     7419
A3     8482
A4    12957
A5    14636
B1    14686
B2    17898
B3    21627
B4    20558
B5    17254
C1    18200
C2    17470
C3    15852
C4    15075
C5    13730
D1    12412
D2    11035
D3     9716
D4     9027
D5     7465
E1     5874
E2     5683
E3     4714
E4     4095
E5     3462
F1     2770
F2     2134
F3     1864
F4     1422
F5     1150
G1      845
G2      600
G3      404
G4      279
G5      240
Name: sub_grade, dtype: int64

In [107]:
# Convert 'sub_grade' to categorical
LoanData = LoanData[pd.isnull(LoanData['sub_grade']) == 0]
LoanData.index = range(len(LoanData))
grades_cat = pd.get_dummies(LoanData['sub_grade'])
LoanData = pd.concat([LoanData, grades_cat], axis=1)

In [108]:
# 3 bits for binary
LoanData.home_ownership.value_counts()

MORTGAGE    152499
RENT        128312
OWN          27180
OTHER          141
NONE            43
ANY              1
Name: home_ownership, dtype: int64

In [109]:
# Convert 'home_ownership' to categorical
home_own = pd.get_dummies(LoanData['home_ownership'])
LoanData = pd.concat([LoanData, home_own], axis=1)

In [110]:
# 4 bits for binary
LoanData.purpose.value_counts()

debt_consolidation    181722
credit_card            62435
home_improvement       18245
other                  16842
major_purchase          7281
small_business          5319
car                     4161
medical                 3355
moving                  2360
wedding                 2066
vacation                1888
house                   1873
educational              325
renewable_energy         304
Name: purpose, dtype: int64

In [111]:
# Convert 'purpose' to categorical
purposes = pd.get_dummies(LoanData['purpose'])
LoanData = pd.concat([LoanData, purposes], axis=1)

In [112]:
# 6 bits for binary conversion
LoanData.addr_state.value_counts().cumsum()

CA     51660
NY     77474
TX    101301
FL    122707
NJ    134256
IL    145564
PA    155848
GA    165640
VA    175134
OH    184589
NC    193013
MI    200450
AZ    207881
MA    215245
MD    222489
WA    229669
CO    236606
MN    242036
MO    246824
NV    251531
CT    255947
OR    260019
AL    263838
WI    267649
IN    271316
LA    274902
SC    278366
TN    281626
KY    284353
OK    287021
UT    289602
KS    292156
AR    294323
HI    296116
NM    297838
WV    299255
NH    300631
RI    301959
DC    302948
MT    303851
AK    304668
DE    305475
WY    306211
SD    306876
MS    307500
VT    308043
NE    308121
ME    308146
ND    308164
IA    308170
ID    308176
Name: addr_state, dtype: int64

In [113]:
# Convert 'addr_state' to categorical
states = pd.get_dummies(LoanData['addr_state'])
LoanData = pd.concat([LoanData, states], axis=1)

In [114]:
# Convert 'verification_status' to categorical
LoanData['verification_status'] = [1 if x == "Source Verified" else 0 for x in LoanData['verification_status']]

In [115]:
# Convert 'application_type' to categorical
LoanData['application_type'] = [0 if x == "INDIVIDUAL" else 1 for x in LoanData['application_type']]

In [116]:
# Convert 'term' to catigorical
LoanData['term'] = [0 if x == " 36 months" else 1 for x in LoanData['term']]

In [117]:
LoanData.isnull().sum()

id                                  0
member_id                           0
loan_amnt                           0
term                                0
int_rate                            0
grade                               0
sub_grade                           0
emp_length                          0
home_ownership                      0
annual_inc                          0
verification_status                 0
issue_d                             0
loan_status                         0
url                                 0
purpose                             0
zip_code                            0
addr_state                          0
dti                                 0
delinq_2yrs                         0
earliest_cr_line                    0
inq_last_6mths                      0
mths_since_last_delinq         169531
mths_since_last_record         267795
open_acc                            0
pub_rec                             0
revol_bal                           0
revol_util  

In [118]:
# Drop nulls in 'mths_since_last_delinq'
LoanData['mths_since_last_delinq'] = [-1 if pd.isnull(x) else x for x in LoanData['mths_since_last_delinq']]

In [119]:
# Drop nulls in 'mths_since_last_record'
LoanData['mths_since_last_record'] = [-1 if pd.isnull(x) else x for x in LoanData['mths_since_last_record']]

In [120]:
# Drop nulls in 'delinq_2yrs'
LoanData['delinq_2yrs'] = [-1 if pd.isnull(x) else x for x in LoanData['delinq_2yrs']]

In [121]:
# Drop nulls in 'mths_since_last_major_derog'
LoanData['mths_since_last_major_derog'] = [-1 if pd.isnull(x) else x for x in LoanData['mths_since_last_major_derog']]

In [122]:
# Drop nulls in 'tot_coll_amt'
LoanData['tot_coll_amt'] = [0 if pd.isnull(x) else x for x in LoanData['tot_coll_amt']]

In [123]:
# Drop nulls in 'tot_cur_bal'
LoanData['tot_cur_bal'] = [0 if pd.isnull(x) else x for x in LoanData['tot_cur_bal']]

In [124]:
# Drop nulls from 'mths_since_last_major_derog'
LoanData['mths_since_last_major_derog'] = [-1 if pd.isnull(x) else x for x in LoanData['mths_since_last_major_derog']]

In [125]:
LoanData.shape

(308176, 142)

In [126]:
# Drop redundant columns
LoanData = LoanData.drop('issue_d', 1)
LoanData = LoanData.drop('earliest_cr_line', 1)
LoanData = LoanData.drop(['grade'], 1)
LoanData = LoanData.drop(['sub_grade'], 1)
LoanData = LoanData.drop(['home_ownership'], 1)
LoanData = LoanData.drop('purpose', 1)
LoanData = LoanData.drop('addr_state', 1)
LoanData = LoanData.drop('url', 1)

In [127]:
LoanData.shape

(308176, 134)

In [128]:
LoanData.head(10)

,id,member_id,loan_amnt,term,int_rate,emp_length,annual_inc,verification_status,loan_status,zip_code,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,mths_since_last_major_derog,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,issue_month,issue_year,first_cr_line,A1,A2,A3,A4,A5,B1,B2,B3,B4,B5,C1,C2,C3,C4,C5,D1,D2,D3,D4,D5,E1,E2,E3,E4,E5,F1,F2,F3,F4,F5,G1,G2,G3,G4,G5,ANY,MORTGAGE,NONE,OTHER,OWN,RENT,car,credit_card,debt_consolidation,educational,home_improvement,house,major_purchase,medical,moving,other,renewable_energy,small_business,vacation,wedding,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
0,1077501,1296599,5000.0,0,10.65,0,24000.0,0,1,860,27.65,0.0,1.0,-1.0,-1.0,3.0,0.0,13648.0,83.7,9.0,-1.0,0,0.0,0.0,0.0,12,2011,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167,2500.0,1,15.27,1,30000.0,1,0,309,1.00,0.0,5.0,-1.0,-1.0,3.0,0.0,1687.0,9.4,4.0,-1.0,0,0.0,0.0,0.0,12,2011,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524,2400.0,0,15.96,0,12252.0,0,1,606,8.72,0.0,2.0,-1.0,-1.0,2.0,0.0,2956.0,98.5,10.0,-1.0,0,0.0,0.0,0.0,12,2011,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178,10000.0,0,13.49,0,49200.0,1,1,917,20.00,0.0,1.0,35.0,-1.0,10.0,0.0,5598.0,21.0,37.0,-1.0,0,0.0,0.0,0.0,12,2011,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1075269,1311441,5000.0,0,7.90,2,36000.0,1,1,852,11.20,0.0,3.0,-1.0,-1.0,9.0,0.0,7963.0,28.3,12.0,-1.0,0,0.0,0.0,0.0,12,2011,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1072053,1288686,3000.0,0,18.64,3,48000.0,1,1,900,5.35,0.0,2.0,-1.0,-1.0,4.0,0.0,8221.0,87.5,4.0,-1.0,0,0.0,0.0,0.0,12,2011,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [83]:
LoanData.issue_year.value_counts().sort_index()

2007      251
2008     1562
2009     4716
2010    11533
2011    20312
2012    50149
2013    86677
2014    86320
2015    46656
Name: issue_year, dtype: int64

In [77]:
LoanData.dtypes

id                              object
member_id                       object
loan_amnt                      float64
term                             int64
int_rate                       float64
emp_length                       int64
annual_inc                     float64
verification_status              int64
loan_status                      int64
zip_code                         int64
dti                            float64
delinq_2yrs                    float64
inq_last_6mths                 float64
mths_since_last_delinq         float64
mths_since_last_record         float64
open_acc                       float64
pub_rec                        float64
revol_bal                      float64
revol_util                     float64
total_acc                      float64
mths_since_last_major_derog    float64
application_type                 int64
acc_now_delinq                 float64
tot_coll_amt                   float64
tot_cur_bal                    float64
issue_month              